In [1]:
# Import packages and libraries
import numpy as np
import random as rnd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from pprint import pprint

from Semi_EM_NB import Semi_EM_MultinomialNB

In [2]:
# Load train and test data set with class labels 
train_Xy = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test_Xy = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
# Convert all text data into tf-idf vectors
vectorizer = TfidfVectorizer()
train_vec = vectorizer.fit_transform(train_Xy.data)
test_vec = vectorizer.transform(test_Xy.data)

In [ ]:
# Divide train data set into labeled and unlabeled data sets
n_train_data = train_vec.shape[0]
split_ratio = 0.3 # labeled vs unlabeled
X_l, X_u, y_l, y_u = train_test_split(train_vec, train_Xy.target, train_size=split_ratio)
print X_l.shape, X_u.shape

(3394, 101631) (7920, 101631)


In [ ]:
# Train Naive Bayes classifier (imported) 
# using both labeled and unlabeled data set
clf = Semi_EM_MultinomialNB(alpha=1e-8) # semi supervised EM based Naive Bayes classifier
clf.fit(X_l, y_l, X_u)

In [ ]:
# Evaluate NB classifier using test data set
pred = clf.predict(test_vec)
print(metrics.classification_report(test_Xy.target, pred, target_names=test_Xy.target_names))

In [ ]:
pprint(metrics.confusion_matrix(test_Xy.target, pred))

In [ ]:
print(metrics.accuracy_score(test_Xy.target, pred))

In [ ]:
b_w_d = X_u>0
type(b_w_d)

In [ ]:
b_w_d.dot(b_w_d.transpose())

In [ ]:
np.concatenate((y_l, y_l), axis=0).shape